In [2]:
from google.colab import drive
drive.mount('/content/drive')

file_pattern = "/content/drive/My Drive/Colab Notebooks/text/crspsample.xlsx"
file_text1 = "/content/drive/My Drive/Colab Notebooks/text/spsample_f94t00.xlsx"
file_text2 = "/content/drive/My Drive/Colab Notebooks/text/spsample_f01t10.xlsx"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import pandas as pd
import numpy as np
import re

pattern = pd.read_excel(file_pattern)
text1 = pd.read_excel(file_text1)
text2 = pd.read_excel(file_text2)

# drop lines containing NaN value
text1 = text1.dropna(subset=['CompanyName', 'CN1'])
text2 = text2.dropna(subset=['CompanyName', 'CN1'])

# reset index (if this step is not conducted, the index will not be continues)
text1 = text1.reset_index(drop = True)
text2 = text2.reset_index(drop = True)

In [58]:
# text1[60:64].isnull()

# drop lines containing NaN value
# text1 = text1.dropna(subset=['CompanyName', 'CN1'])

text1[60:64]

,sp_id,Year,CompanyName,CN1,matched
60,1994_10051,1994,"CEREAL FOOD PROCESSORS, INC.","CEREAL FOOD PROCESSORS, INC.",NaN
61,1994_10053,1994,"CERMETEK MICROELECTRONICS, INC.","CERMETEK MICROELECTRONICS, INC.",NaN
62,1994_10054,1994,CERNER CORP.,CERNER CORP.,1.0
63,1994_10055,1994,CEROMET INC.,CEROMET INC.,NaN


In [12]:
# calculate the similarity of two company names
import difflib
 
def similarity(s1, s2):
    return difflib.SequenceMatcher(None, s1, s2).quick_ratio()

similarity("A T C GROUP SERVICES INC", "AUTOGRAPHIC SERVICES, INC. ")

# similarity("CERPROBE CORP", "CERPROBE CORP")

0.8627450980392157

In [ ]:
names = []
sw_list = []
n1 = len(text1.axes[0])
n2 = len(text2.axes[0])
m = len(pattern.axes[0])
count = 0

# match p to all companies in text1
for row_ in range(n1):
    year = int(text1['Year'][row_])
    txt = text1['CN1'][row_]
    orig = text1['matched'][row_]

    # mark that there must be a match for this company
    # if a match is founded later, meaning that this notation is correct
    # if not, meaning that this notation is wrong
    if orig == 1:
        flag = 0
        print("Already Matched for text={txt}".format(txt=txt))

    for row in range(m):
        date_start = pattern['date'][row]
        date_start = int(date_start.year) - 2
        date_end = pattern['nameendt'][row]
        date_end = int(date_end.year) + 2
        pat = pattern['comnam'][row]

        sim = similarity(pat, txt)

        if year >= date_start and year <= date_end and sim >= 0.87:
            if orig == 1:  # already matched
                flag = 1  # the match is founded
                print("   ~ pattern={pat}, similarity={sim}".format(pat=pat, sim=sim))
            else:
                print("text={txt} ~ pattern={pat}, similarity={sim}".format(pat=pat, txt=txt, sim=sim))
                text1.loc[text1.CN1==txt,'matched'] = 1
                
      
    if orig == 1 and flag == 0:  # 如果text1中标注了"必存在match"，说明我的算法有问题，找不到它应找到的匹配
        count += 1
        print("algorithm error!")

        
print(count)

Already Matched for text=AERCO INTERNATIONAL, INC. 
   ~ pattern=VARCO INTERNATIONAL INC, similarity=0.8979591836734694
   ~ pattern=ESARCO INTERNATIONAL INC, similarity=0.92
   ~ pattern=ESARCO INTERNATIONAL INC, similarity=0.92
   ~ pattern=ESARCO INTERNATIONAL INC, similarity=0.92
   ~ pattern=CORVAS INTERNATIONAL INC, similarity=0.88
   ~ pattern=ROMAC INTERNATIONAL INC, similarity=0.8979591836734694
   ~ pattern=BONE CARE INTERNATIONAL INC, similarity=0.9056603773584906
   ~ pattern=ORTEC INTERNATIONAL INC, similarity=0.8979591836734694
Already Matched for text=CENTURY GEOPHYSICAL CORP.
   ~ pattern=SUPERMAC TECHNOLOGY INC, similarity=0.875
Already Matched for text=CENTURY INDUSTRIES CORP.
   ~ pattern=CYBERNETICS PRODUCTS INC, similarity=0.875
Already Matched for text=CENTURY LABORATORIES, INC.
   ~ pattern=ROYCE LABORATORIES INC, similarity=0.875
   ~ pattern=ROYCE LABORATORIES INC, similarity=0.875
   ~ pattern=AURORA ELECTRONICS INC, similarity=0.875
Already Matched for text=C

In [67]:
text1.tail(10)

,sp_id,Year,CompanyName,CN1,matched
449542,2000_9990,2000,BRUNER WATER TREATMENT SYSTEMS,BRUNER WATER TREATMENT SYSTEMS,NaN
449543,2000_9991,2000,BRUNING PAINT COMPANY,BRUNING PAINT COMPANY,NaN
449544,2000_9992,2000,BRUNNER ERBEN AG,BRUNNER ERBEN AG,NaN
449545,2000_9993,2000,BRUNNER INC. (Div. Brunner Mfg. Inc.),BRUNNER INC.,NaN
449546,2000_9994,2000,"BRUNNER & LAY, INC.","BRUNNER & LAY, INC.",NaN
449547,2000_9995,2000,"BRUNNER MAZEL, INC. (Div. Taylor & Francis, Inc.)","BRUNNER MAZEL, INC.",NaN
449548,2000_9996,2000,BRUNNER MONO pie,BRUNNER MONO pie,NaN
449549,2000_9997,2000,BRUNNER NEWS AGENCY,BRUNNER NEWS AGENCY,NaN
449550,2000_9998,2000,"BRUNNQUELL IRON WORKS, INC.","BRUNNQUELL IRON WORKS, INC.",NaN
449551,2000_9999,2000,BRUNO MACHINERY CORP.,BRUNO MACHINERY CORP.,NaN


In [ ]:
text1.to_excel(r'/content/drive/My Drive/Colab Notebooks/text/matched1.xlsx', index = None, header=True)